# Agents in Action Workshop (AWS)

<div>
    <center>
        <img src="thoughtworks-logo.png" width="400" />
    </center>
</div>

Credits: [Thoughtworks, 2025](https://Thoughtworks.com)

[Ricardo Teixara](mailto:ricardo.teixera@thoughtworks.com), [Ben O'Mahony](
ben.omahony@thoughtworks.com), [Yuvaraj Birari](mailto:Yuvaraj.Birari@thoughtworks.com), [Sebastian Werner](mailto:sebastian.werner@thoughtworks.com), [Danilo Sato](mailto:danilo.sato@thoughtworks.com) & [Kalyan Muthiah](mailto:kmuthiah@thoughtworks.com)

We use pydantic AI to save us from some of the heavy lifting, here are the docs for[Pydantic](https://ai.pydantic.dev/agents/).  
Alternatives include [LangChain](https://www.langchain.com/langchain), and many more of variable maturity.

<div>
    <center>
        <img src="agent-diagram.png" width="800" />
    </center>
</div>


# Setup

Install pydantic dependency to access bedrock.

In [1]:
pip install "pydantic-ai-slim[bedrock, mcp]"

Note: you may need to restart the kernel to use updated packages.


Test whether the API is working by listing the available GenAI models.

In [2]:
import logging
import json
import boto3
from botocore.exceptions import ClientError


def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        return models

    except ClientError:
        print("Couldn't list foundation models.")
        raise


bedrock_client = boto3.client(service_name="bedrock")

fm_models = list_foundation_models(bedrock_client)
print(f"Found {len(fm_models)} models:")

for model in fm_models:
    print(f"Model: {model['modelName']}")
    # print(json.dumps(model, indent=2))

Found 26 models:
Model: Titan Text G1 - Express
Model: Titan Text G1 - Express
Model: Titan Text G1 - Lite
Model: Titan Text G1 - Lite
Model: Titan Embeddings G1 - Text
Model: Titan Embeddings G1 - Text
Model: Titan Multimodal Embeddings G1
Model: Titan Multimodal Embeddings G1
Model: Titan Embeddings G2 - Text
Model: Rerank 1.0
Model: Nova Pro
Model: Nova Lite
Model: Nova Micro
Model: Claude
Model: Claude
Model: Claude 3 Sonnet
Model: Claude 3 Haiku
Model: Claude 3.5 Sonnet
Model: Claude 3.7 Sonnet
Model: Claude Sonnet 4
Model: Embed English
Model: Embed Multilingual
Model: Rerank 3.5
Model: Pixtral Large (25.02)
Model: Llama 3.2 1B Instruct
Model: Llama 3.2 3B Instruct


Fix the notebook vs. MCP async communication:

In [3]:
import nest_asyncio
nest_asyncio.apply()

Establish the model - Claude 3 Sonnet is good enough:

In [4]:
from pydantic_ai import Agent
from pydantic_ai.models.bedrock import BedrockConverseModel

model = BedrockConverseModel('anthropic.claude-3-sonnet-20240229-v1:0')
agent = Agent(model)

# First use of the model

Test that our model is working:

In [5]:
o = agent.run_sync("What is the UK city known as the 'capital of the north'?").output
print(o)

Manchester is often referred to as the 'capital of the north' in the United Kingdom.

Here are some key points about Manchester's status as the northern capital:

- Manchester has a large population (over 500,000 in the city itself and over 2.8 million in Greater Manchester) making it the major urban center in the north of England.

- It has been an important industrial and economic hub since the 19th century during the Industrial Revolution.

- Major sectors include media, digital industries, manufacturing, engineering, and financial services.

- Manchester has two large universities - the University of Manchester and Manchester Metropolitan University.

- It has an extensive transport network with an international airport and rail connections to other major cities.

- Manchester is a cultural center with museums, theaters, music venues and premier league sports teams.

- Politically and historically, Manchester has played a leading role in the north, hosting major events and being a 

Great! It's working.

Now let's use structured output and save it for later:

In [6]:
from pydantic import BaseModel

class location(BaseModel):
    city: str
    country: str

agent = Agent(model, output_type=location)

location_result = agent.run_sync("What is the UK city known as the 'capital of the north'?").output
print(f"{location_result.city}, {location_result.country}")

Manchester, United Kingdom


# Set up our mock backend services

We setup a mock customer database used by a mock support service.  
The service will then be used by our agent to get information it needs.

Our mock customer database. Just one customer (#123) named Alex with one pending order (#978):

In [7]:
from dataclasses import dataclass

class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def customer_name(cls, *, id: int) -> str | None:
        if id == 123:
            return 'Alex Ferguson'
        else:
            raise ValueError('Customer not found')

    @classmethod
    async def customer_order(cls, *, id: int, include_pending: bool) -> float:
        if id == 123 and include_pending:
            return {'order_id': 987, 'item': 'Really shady Sunglasses', 'quantity': 1, 'price': 23.42, 'ordered_on': '2025-09-14 23:42:05'}
        else:
            raise ValueError('Customer not found')


Set up the dependencies for the agents, one of which is the database we setup before:

In [8]:
from dataclasses import dataclass
from pydantic import Field

@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportOutput(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    urgency: int = Field(description='Urgency level of query', ge=0, le=10)

@dataclass
class OrderStatusDependencies:
    customer_id: int
    db: DatabaseConn


class OrderStatusOutput(BaseModel):
    expected_delivery: str = Field(description='Expected delivery date for pending order')
    order_id: int = Field(description='order_id retrieved from `db`')


# Define our agents

Construct our support and order agents using our model, with the above defined dependencies and structured suppport output:

In [9]:
from pydantic_ai import Agent, RunContext

# Our orchestrator agent (calls other agent via tool)
support_agent = Agent(
    model=model,
    deps_type=SupportDependencies,
    output_type=SupportOutput,
    system_prompt=(
        """
        You are a support agent in our market leading company, give the
        customer support and judge the urgency level of their query.
        Reply using the customer's name.
        Be friendly and supportive.
        Use the `order_tool` to find out the order delivery date using the `customer_id`.
        Apologise if the order is past its expected delivery date.
        Find the current date using `current_date_tool`.
        """
    ),
)

# An executer agent (action is querying the database)
order_agent = Agent(
    model=model,
    deps_type=OrderStatusDependencies,
    output_type=OrderStatusOutput,
    system_prompt=(
        """
        You are a order agent in charge of customers orders.
        Retrieve the customer's order using `query_database_tool` with the `customer_id`.
        The expected delivery date is the next weekday after adding 7 days to the `ordered_on` date.
        Reply with the `order_id` number and expected delivery date for the user pending order.
        """
    ),
)

Define a tool for the `support_agent` to get the customer order using the `order_agent` we defined above.

<div>
    <center>
        <img src="agent-diagram-agent-to-agent.png" width="800" />
    </center>
</div>

The `order_agent` then uses the `query_database_tool` that calls the mock `db` connection to retrieve the order.  
We also define a `current_date_tool` so the agent knows what is the current date.

In [10]:
from datetime import datetime

# Uses the order_agent to find the customer orders
@support_agent.tool
async def order_tool(
    ctx: RunContext[SupportDependencies], include_pending: bool) -> str:
    id = ctx.deps.customer_id,
    include_pending=include_pending,

    r = await order_agent.run(
        f'Please retrieve order for client {id}', deps=OrderStatusDependencies(customer_id=ctx.deps.customer_id, db=ctx.deps.db)
    )
    return r.output

# Connects to the database and gets the pending orders for the customer
@order_agent.tool
async def query_database_tool(
    ctx: RunContext[OrderStatusDependencies], include_pending: bool) -> str: 
    order = await ctx.deps.db.customer_order(
        id=ctx.deps.customer_id,
        include_pending=include_pending,
     )
    return f"id {order['order_id']:d}, of {order['quantity']} '{order['item']}' for {order['price']:.2f} ordered on {order['ordered_on']}"

# Returns the current date
@support_agent.tool
async def current_date_tool(
    ctx: RunContext[SupportDependencies]) -> str:
    
    return datetime.today().strftime('%Y-%m-%d')

# Customer name in the system prompt
@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}, their customer_id is {ctx.deps.customer_id}"

Test the order agent:

In [11]:
o = order_agent.run_sync('Tell me my pending order_id?', deps=OrderStatusDependencies(customer_id=123, db=DatabaseConn())).output

print(f"Order {o.order_id} is pending, with an expected delivery date of {o.expected_delivery}.") # note the structured output

Order 987 is pending, with an expected delivery date of 2025-09-22.


# Running our agents

We ask if the order is going to be delivered and the agent should tell us it is late:

In [12]:
deps = SupportDependencies(customer_id=123, db=DatabaseConn())

o = support_agent.run_sync('When will my order be delivered?', deps=deps).output

print(o.support_advice)

Hi Alex, I looked into your order details with customer ID 123. The expected delivery date for order #987 was 2025-09-22, which is already a couple of days past. Thank you for your patience and I sincerely apologize for the delay. I've escalated this issue to our logistics team to investigate and prioritize your order's shipment. Please feel free to reach out again if you have any other questions or concerns!


# Using a simple MCP server

<div>
    <center>
        <img src="agent-diagram-mcp.png" width="800" />
    </center>
</div>

Create an very very simple MPC server that shows the weather:

In [13]:
cat mcp-weather.py # contents of mcp-weather.py. Should be run in the terminal.

import httpx
import logging
from mcp.server.fastmcp import FastMCP

mcp_server = FastMCP()

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@mcp_server.tool()
async def fetch_weather(city: str) -> str:
    """Fetch current weather for a city"""
    
    #async with httpx.AsyncClient() as client:
    #    response = await client.get(f"https://wttr.in/{city}?format=%C+%t")
    #    return response.text

    # fake response so it is always bad weather!
    response = f"Weather for {city} is -10 degrees Celsius and snowing."
    logger.info(response)
    
    return response


if __name__ == '__main__':
    
    mcp_server.run(transport='streamable-http')


Register the MCP server:

In [14]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.mcp import MCPServerStreamableHTTP

mcp_weather_server = MCPServerStreamableHTTP('http://localhost:8000/mcp')

Add to our `support_agent` prompt so it also uses the weather MCP server. Also add a `current_location_tool` to find the current location:

In [15]:

# Returns the current date
@support_agent.tool
async def current_location_tool(
    ctx: RunContext[SupportDependencies]) -> str:
    
    return f"Current city is {location_result.city}"

# Extra instructions to use the weather MCP server
@support_agent.instructions
def use_mcp_server() -> str:  
    return (
        """Find the current city using `current_location_tool`.
        Fetch the weather using the usign the `fetch_weather` tool for the current city.
        If the order is late, and there is bad weather, make an excuse about the bad weather.
        Always tell the weather conditions, including temperature and current city, to the customer.""")

Now query the agent, adding the MCP to the toolset.

In [16]:
from pydantic_ai.toolsets import FunctionToolset

o = support_agent.run_sync('When will my order be delivered?', deps=deps, toolsets=[mcp_weather_server]).output

print(o.support_advice)

Hi Alex Ferguson, I apologize that your order #987 is delayed due to the severe winter weather conditions here in Manchester with temperatures as low as -10C and heavy snow. Your order was expected to be delivered on 2025-09-22 but there has been a delay because of the difficult travel conditions. The current date is 2025-09-24. Please be assured we are working hard to get your order to you as soon as the weather improves. In the meantime, I appreciate your patience and understanding regarding this weather-related delay.
